In [1]:
%load_ext autoreload
%autoreload 2

import dataclasses

import torch
from torch import nn

from src.network_analysis import count_parameters
from src.networks.fnn import *
from src.networks.transformer_encoder import *
from src.networks.skip_connection import *

TypeError: Too many arguments for typing.Callable[..., ~MODULE_TYPE]; actual 2, expected 1

In [19]:
FNN.self_normalizing(
    1, 
    [2, 5], 
    3, 
    activate_last_layer=True, 
    # dropout_p=0.1, 
    # dropout_last_layer=True
)

FNN(
  (fnn): Sequential(
    (0): Linear(in_features=1, out_features=2, bias=True)
    (1): SELU()
    (2): Linear(in_features=2, out_features=5, bias=True)
    (3): SELU()
    (4): Linear(in_features=5, out_features=3, bias=True)
    (5): SELU()
  )
)

In [35]:
transformer = TransformerEncoder(
    num_layers=8,
    num_features=32,
    attention_num_heads=4,
    feedforward_provider=lambda num_features: FNN(
        input_size=num_features, hidden_sizes=[256], output_size=num_features
    ),
    residual_connection_weight_affine=True,
)
print(count_parameters(transformer))

# nn_transformer = nn.TransformerEncoder(
#     nn.TransformerEncoderLayer(
#         d_model=512,
#         nhead=8,
#     ),
#     num_layers=6
# )
# print(count_parameters(nn_transformer))

# count_parameters(transformer) / count_parameters(nn_transformer)

168704


18914304

1.0